<a href="https://colab.research.google.com/github/PanchenkoYehor/gan-pok/blob/main/ProGAN%2B_Driver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import/Connect to Google Drive

In [1]:
import datetime
from google.colab import drive
import IPython
from pathlib import Path
from PIL import Image
import sys
import tensorflow as tf
import tensorflow_datasets as tfds
import time

drive.mount('/content/drive/')
sys.path.insert(1, '/content/drive/MyDrive/progan-plus')

from create_strategy import create_strategy
import main_training


Mounted at /content/drive/


# Train

In [2]:
class Visualizer:
    def __init__(self, new_display_interval=10):
        self.display_count = 0
        self.new_display_interval = new_display_interval
        self.vis_display = IPython.display.display(display_id=True)

    def on_visualization(self, image):
        self.display_count += 1
        image = Image.fromarray(image.numpy())
        if self.display_count % self.new_display_interval == 0:
            self.vis_display = IPython.display.display(image, display_id=True)
        else:
            self.vis_display.update(image)

In [3]:
strategy = create_strategy()

In [4]:
tf.keras.mixed_precision.set_global_policy('mixed_bfloat16')
# tf.keras.mixed_precision.set_global_policy('mixed_float16')

init_mnist_arguments = [
    'init',
    '--visualization-grid-size',
    '30',
    '60',
    '--resolution=32',
    '--replica-batch-size=32',
    f'--epoch-sample-count={1024*128}',
    f'--total-sample-count={25*1024*1024}',
    '--visualization-smoothing-sample-count=1000',
    '--checkpoint-interval=10',
    '--disable-horizontal-flip-data-augmentation',
    '--latent-size=3',
    'gs://bk-mnist/32x32/*.tfrecord',
    '/content/drive/MyDrive/progan-plus-output/mnist']

resume_mnist = [
    'resume',
    '/content/drive/MyDrive/progan-plus-output/mnist']

init_ffhq_arguments = [
    'init',
    '--visualization-grid-size',
    '4',
    '8',
    '--resolution=512',
    '--replica-batch-size=8',
    f'--epoch-sample-count={1024*16}',
    f'--total-sample-count={25*1024*1024}',
    '--visualization-smoothing-sample-count=10000',
    '--checkpoint-interval=10',
    '--latent-size=512',
    'gs://bk-ffhq/512x512/*.tfrecord',
    '/content/drive/MyDrive/progan-plus-output/ffhq']

resume_ffhq = [
    'resume',
    '/content/drive/MyDrive/progan-plus-output/ffhq']

visualizer = Visualizer()
def on_visualization(image):
    visualizer.on_visualization(image)

main_training.main(
    raw_arguments=init_ffhq_arguments,
    visualization_callback=on_visualization,
    strategy=strategy)

  1/256 [..............................] - ETA: 8:17:12 - d_loss: 1.4297 - d_real: 0.5632 - d_fake: 0.8665 - d_grad_reg: 0.0321 - g_loss: 1.2497 - epoch: 0.0000e+00

 48/256 [====>.........................] - ETA: 3:54 - d_loss: 0.7604 - d_real: 0.5037 - d_fake: 0.2566 - d_grad_reg: 0.3282 - g_loss: 7.2642 - epoch: 0.0000e+00

KeyboardInterrupt: ignored